In [1]:
!pip install torchdiffeq

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
import pandas as pd
import numpy as np
import torch
from torchdiffeq import odeint

In [3]:
##############################################################################
# Utility Functions
##############################################################################

In [4]:
import torch

def linear_interp1d(x, y, x_new):
    """
    Perform linear interpolation on 1D data.

    Args:
    x (torch.Tensor): The original x-coordinates (time or space), must be in increasing order.
    y (torch.Tensor): The original y-values (values that correspond to x).
    x_new (torch.Tensor): The new x-coordinates where interpolation is desired.

    Returns:
    torch.Tensor: Interpolated values at x_new.
    """
    # Edge case handling
    if torch.any(x_new < x[0]) or torch.any(x_new > x[-1]):
        raise ValueError("x_new values are outside the range of x")

    # Find indices of segments containing x_new points
    idxs = torch.searchsorted(x, x_new, right=True) - 1
    idxs = torch.clamp(idxs, 0, len(x)-2)  # Handle edge case for last point

    # Get corresponding original points and differences
    x1, x2 = x[idxs], x[idxs + 1]
    y1, y2 = y[idxs], y[idxs + 1]
    dx, dy = x2 - x1, y2 - y1

    # Calculate the weights for interpolation
    weights = (x_new - x1) / dx

    # Perform interpolation
    y_new = y1 + weights * dy

    return y_new

In [5]:
# Gathering some Data from a Coffee Roaster

In [6]:
%cd /content/drive/MyDrive/Coffee_Thesis/ODE model

/content/drive/MyDrive/Coffee_Thesis/ODE model


In [7]:
# Read the csv
df = pd.read_csv('1_ROAST_29_01_2021_00_46_43_P16.csv')
# Collect the relevant data
df_inputs = df[['Actual value T1 [°C]', 'Air speed meter fSCP [m/sec] ','Time Stamp', 'Actual value T2 [°C]']].copy(deep = True)
# Format the time stamps
df_inputs['Time Stamp'] = pd.to_datetime(df_inputs['Time Stamp'], format='%d/%m/%Y %H:%M:%S')

# I don't end up using the below, but it is there anyways for now
# Calculate time differences
df_inputs['duration'] = df_inputs['Time Stamp'].diff()
# Replace the first NaT with 0 timedelta
df_inputs['duration'].iloc[0] = pd.Timedelta(seconds = 0)
# Convert duration to total seconds as a float
df_inputs['duration'] = df_inputs['duration'].dt.total_seconds()

<ipython-input-7-bc131a8d1ae5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inputs['duration'].iloc[0] = pd.Timedelta(seconds = 0)


In [8]:
###############################################################################
# MODEL OF THE COFFEE ROASTING DYNAMICS
###############################################################################

class CoffeeRoastingModel(torch.nn.Module):
  def __init__(self, df_inputs):
    super().__init__()
    # Initialize parameters as torch.nn.Parameters
    # Fixed Parameters:
    self.A = torch.nn.Parameter(torch.tensor(116200 * 1000, dtype=torch.float32)) # J/kg
    self.cm = torch.nn.Parameter(torch.tensor(0.418 * 1000, dtype=torch.float32))  # J/(kg °C)
    self.cw = torch.nn.Parameter(torch.tensor(5 * 1000, dtype=torch.float32))  # J/(kg °C)
    self.Db = torch.nn.Parameter(torch.tensor(7.65e-3, dtype=torch.float32)) # m
    self.Ha_R = torch.nn.Parameter(torch.tensor(5500, dtype=torch.float32))  # K (this is Ha/R)
    self.Het = torch.nn.Parameter(torch.tensor(232 * 1000, dtype=torch.float32))  # J/kg
    self.k1 = torch.nn.Parameter(torch.tensor(4.32e-9, dtype=torch.float32))
    self.k2 = torch.nn.Parameter(torch.tensor(9889, dtype=torch.float32))
    self.Kt = torch.nn.Parameter(torch.tensor(0.01, dtype=torch.float32))   # 1/s
    self.mb = torch.nn.Parameter(torch.tensor(1.5e-4, dtype=torch.float32))  # kg
    self.λ = torch.nn.Parameter(torch.tensor(2790 * 1000, dtype=torch.float32))    # J/kg
    # Scalable Parameters:
    self.Mb = torch.nn.Parameter(torch.tensor(120, dtype=torch.float32))     # kg #Also 320 kg
    self.Dch = torch.nn.Parameter(torch.tensor(1.24, dtype=torch.float32))   # m #Also 1.90
    self.Hflap = torch.nn.Parameter(torch.tensor(0.3, dtype=torch.float32))  # m
    self.Lch = torch.nn.Parameter(torch.tensor(1.335, dtype=torch.float32))  # m #Also 2.04
    self.Mm = torch.nn.Parameter(torch.tensor(2000, dtype=torch.float32))    # kg #also 7000
    self.Sflap = torch.nn.Parameter(torch.tensor(0.1, dtype=torch.float32))  # m
    self.Pbm = torch.nn.Parameter(torch.tensor(0.5793, dtype=torch.float32))  # Hypothetical value for bean-metal contact area (this is a percentatge and it relies on the amount of beans and the geometry of the roaster)
    # heat transfer coefficients
    self.hgm = torch.nn.Parameter(torch.tensor(105, dtype=torch.float32)) #0.01  # W/(m^2*K) Hypothetical value (although these could change as a function of something else)
    self.hbm = torch.nn.Parameter(torch.tensor(30, dtype=torch.float32)) #0.0254  # W/(m^2*K) Hypothetical value (although these could change as a function of something else)

    # heat transfer areas based on only on parameters (time invariant)
    self.Agm = self.calc_Agm()
    self.Ab = self.calc_Ab()
    self.Agb = self.calc_Agb()
    self.Abm = self.Ab.clone() * self.Pbm.clone()

    # Input functions definitions
    # Convert the DataFrame columns directly to tensors within the class
    self.times = torch.tensor(df_inputs.index.values, dtype=torch.float32)
    self.Gg_values = torch.tensor(df_inputs['Air speed meter fSCP [m/sec] '].values, dtype=torch.float32)
    self.Tgi_values = torch.tensor(df_inputs['Actual value T1 [°C]'].values, dtype=torch.float32)

    # Calculate slopes (finite differences) for Gg and convert to tensor
    dGgdt = np.diff(self.Gg_values.numpy()) / np.diff(self.times.numpy())
    dGgdt = np.append(dGgdt, dGgdt[-1])  # Using backward fill for the last element
    self.dGgdt = torch.tensor(dGgdt, dtype=torch.float32)

  def linear_interp1d(self, x, y, x_new):
      # Assuming x_new is a scalar for simplicity
      idxs = torch.searchsorted(x, x_new, right=True) - 1
      idxs = torch.clamp(idxs, 0, len(x) - 2)

      x1, x2 = x[idxs], x[idxs + 1]
      y1, y2 = y[idxs], y[idxs + 1]
      dx, dy = x2 - x1, y2 - y1

      # Weight for linear interpolation
      weight = (x_new - x1) / dx
      y_new = y1 + weight * dy
      return y_new

  def input_Gg(self, t):
      t_tensor = torch.tensor([t], dtype=torch.float32) if not torch.is_tensor(t) else t
      return self.linear_interp1d(self.times, self.Gg_values, t_tensor)

  def input_Tgi(self, t):
      t_tensor = torch.tensor([t], dtype=torch.float32) if not torch.is_tensor(t) else t
      return self.linear_interp1d(self.times, self.Tgi_values, t_tensor)

  def input_dGgdt(self, t):
      t_tensor = torch.tensor([t], dtype=torch.float32) if not torch.is_tensor(t) else t
      return self.linear_interp1d(self.times, self.dGgdt, t_tensor)

  # Set up other intermediary functions
  def calc_Ab(self):
    Db_squared = (self.Db**2).detach()  # Detach from the current graph
    Db_squared.requires_grad_(True)  # Manually enable gradient tracking
    result = (self.Mb / self.mb) * torch.pi * Db_squared
    return result

  def calc_Agb(self):
    return self.Ab * (1 - self.Pbm)

  def calc_Agm(self):
    return torch.pi * self.Dch * (self.Lch + (self.Hflap * self.Lch) / self.Sflap + self.Dch / 2)

  def calc_F(self, he):
    return self.hgm * self.Agm / (he * self.Agb)

  def calc_he(self, X):
    return 0.49 - 0.443 * torch.exp(-0.206 * X)

  # Function for gas specific heat capacity (from paper):
  def cg(self, T):
    alpha = torch.tensor([1.0839e3, -7.2075e-1, 2.1034e-3, -2.3267e-6, 1.3621e-9, -4.1550e-13, 5.3091e-17], dtype=torch.float32)
    powers = torch.tensor([i for i in range(7)], dtype=torch.float32)
    return torch.sum(alpha * (T + 273.15)**powers)

  # Function for outlet gas temperature (from paper):
  def calc_Tgo(self, x, Gg, Tgi):
    Tb, Tm, X, He, Ta = x
    he_val = self.calc_he(X).clone()
    F_val = self.calc_F(he_val).clone()
    return Tgi - ((Tb + F_val * Tm) / (1 + F_val)) * (1 - torch.exp(-he_val * self.Agb * (1 + F_val) / (Gg * self.cg(Tgi))))

  # This is the main calculation for the ODE
  def dynamics(self, t, x):
    # Debugging
    # #print(f"Time: {t.item()}, State: {x}")
    # print("____Before forward pass____:")
    # for name, param in self.named_parameters():
    #     # print(f"{name}: version {param._version}")
    #     print(f"{name}: temp requires_grad: {param.requires_grad}")

    Tb, Tm, X, He, Ta = x
    # Input functions evaluations
    Tgi = self.input_Tgi(t)   # Inlet gas temperature
    Gg = self.input_Gg(t)     # Input gas rates
    dGgdt = self.input_dGgdt(t)

    # Calculating specific heat capacities
    cs = 1.099 + 0.007 * Tb
    cb = (cs + self.cw * X) / (1 + X)

    # Heat transfer coefficients:
    he = self.calc_he(X)

    # Calculate Tgo:
    Tgo = self.calc_Tgo(x, Gg, Tgi)

    # Calculate heat transfer rates:
    F = self.calc_F(he)
    Qgb = Gg * self.cg(Tgi) * (Tgi - Tgo)
    Qgm = F * (he * self.Agb * (Tb - Tm) + Qgb) / (1 + F)
    Qbm = self.hbm * self.Abm * (Tm - Tb)

    # Calculate moisture loss and exothermic heat:
    dXdt = -self.k1 / self.Db**2 * torch.exp(-self.k2 / (Tb + 273.15))
    Qr = self.A * ((self.Het - He) / self.Het) * torch.exp(-self.Ha_R / (Tb + 273.15))

    # Bean temperature dynamics:
    Mbd = self.Mb / (1 + X)
    dTbdt = (Qgb - Qgm + Qbm + Mbd * (Qr + self.λ * dXdt)) / (Mbd * (1 + X) * cb)

    # Metal temperature dynamics:
    dTmdt = (Qgm - Qbm) / (self.Mm * self.cm)

    # Sensor dynamics for measured bean temperature
    dTadt = self.Kt * (Tb - Ta)

    # Energy conservation check
    heat_out_of_gas = Qgb + Qgm
    heat_change_in_gas = Gg * self.cg(Tgi) * (Tgi - Tgo)

    # Debugging
    #print(f' ____start______\ Gg {Gg}\n Tgi {Tgi}\n Tgo {Tgo} \n Qr {Qr}\n Qgb {Qgb}\n Qgm {Qgm}\n Qbm {Qbm}\n He {He}\n Agb {self.Agb}\n Agm {self.Agm}\n Abm {self.Abm}\n cs {cs}\n cb {cb}\n cg {self.cg(Tgi)}\n cm {self.cm}\n he {he}\n hgm {self.hgm}\n hbm {self.hbm}\n ')
    # print(f'____QR____\n A {self.A}\n Het {self.Het}\n He {He}\n Ha_R {self.Ha_R}\n Tb {Tb}')
    # print(f"Derivatives at Time {t.item()}: {[dTbdt, dTmdt, dXdt, Qr, dTadt]}")
    # input()
    # print("___After forward pass:___")
    # for name, param in self.named_parameters():
    #     print(f"{name}: version {param._version}")
    #     print(f"{name} temp requires_grad: {param.requires_grad}")

    return torch.stack([dTbdt, dTmdt, dXdt, Qr, dTadt])

  def forward(self, t, x):
    # This is what is called by the ODE solver
    return self.dynamics(t, x)



In [9]:
###############################################################################
# Fitting the parameters to the exhaust data
###############################################################################

In [11]:
torch.autograd.set_detect_anomaly(True)

model = CoffeeRoastingModel(df_inputs) # initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_function = torch.nn.MSELoss()

# Initial conditions:
Tb0 = 30     # Initial bean temperature
Tm0 = 185    # Initial metal temperature
X0 = 0.90     # Initial moisture content
He0 = 0      # Initial exothermic heat
Ta0 = 142.7     # Initial measured bean temperature

# Initial conditions for state variables
x0 = [Tb0, Tm0, X0, He0, Ta0]
initial_state = torch.tensor(x0, dtype=torch.float32)

# Input functions definitions
# Note that this is also done in the CoffeeRoastingModel class
# Convert the DataFrame columns directly to tensors within the class
times = torch.tensor(df_inputs.index.values, dtype=torch.float32)
Gg_values = torch.tensor(df_inputs['Air speed meter fSCP [m/sec] '].values, dtype=torch.float32)
Tgi_values = torch.tensor(df_inputs['Actual value T1 [°C]'].values, dtype=torch.float32)
Tgo_measured = torch.tensor(df_inputs['Actual value T2 [°C]'].values, dtype=torch.float32)

epochs = range(100)
final_Tgo_predictions = None

for epoch in epochs:


  # Integrate the ODE to get the states
  states_over_time = odeint(model, initial_state, times)

  # Calculate Tgo for each time point
  Tgo_predictions = torch.stack([model.calc_Tgo(state, Tgi_val, Gg_val) for state, Tgi_val, Gg_val in zip(states_over_time, Tgi_values, Gg_values)])

  # Calculate the loss
  # Assuming Tgo_predictions is the tensor used to compute the loss
  #print("Tgo_predictions requires_grad:", Tgo_predictions.requires_grad)
  loss = loss_function(Tgo_predictions, Tgo_measured)
  optimizer.zero_grad()
  loss.backward(retain_graph=True)

  print(f"__Versions at the end of Epoch {epoch}__")
  for name, param in model.named_parameters():
    print(f"{name}: version {param._version}")

  optimizer.step() # Update parameters

  print(f"__Versions at the beginning of Epoch {epoch+1}__")
  for name, param in model.named_parameters():
    print(f"{name}: version {param._version}")
  print(f'Epoch {epoch}: Loss = {loss.item()}')

  # Store the last set of predictions
  if epoch == epochs[-1]:
    final_Tgo_predictions = Tgo_predictions.detach().numpy()

# After Training print out the parameters
for name, param in model.named_parameters():
  print(f'{name}: {prarm.data}')

__Versions at the end of Epoch 0__
A: version 0
cm: version 0
cw: version 0
Db: version 0
Ha_R: version 0
Het: version 0
k1: version 0
k2: version 0
Kt: version 0
mb: version 0
λ: version 0
Mb: version 0
Dch: version 0
Hflap: version 0
Lch: version 0
Mm: version 0
Sflap: version 0
Pbm: version 0
hgm: version 0
hbm: version 0
__Versions at the beginning of Epoch 1__
A: version 1
cm: version 1
cw: version 1
Db: version 1
Ha_R: version 1
Het: version 1
k1: version 1
k2: version 1
Kt: version 1
mb: version 1
λ: version 1
Mb: version 1
Dch: version 1
Hflap: version 1
Lch: version 1
Mm: version 1
Sflap: version 1
Pbm: version 1
hgm: version 1
hbm: version 1
Epoch 0: Loss = 166537.359375


/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: Error detected in DivBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/lib/

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []] is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [ ]:
import matplotlib.pyplot as plt

# Convert tensors to numpy arrays if they aren't already (assuming they're in PyTorch tensors)
final_Tgo_predictions = final_Tgo_predictions.detach().cpu().numpy()  # Detach and move to cpu if needed
Tgo_measured = Tgo_measured.detach().cpu().numpy()  # Detach and move to cpu if needed

# Create a plot
plt.figure(figsize=(10, 6))
plt.plot(times, final_Tgo_predictions, label='Predicted Tgo', marker='o', linestyle='-')
plt.plot(times, Tgo_measured, label='Measured Tgo', marker='x', linestyle='--')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Outlet Gas Temperature (Tgo)')
plt.title('Comparison of Predicted and Measured Outlet Gas Temperatures')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()